In [3]:
import sys
sys.path.append('../')
import importlib
import models.transunet as vit
import models.decoder_layers as decoder_layers
import models.utils as ut
import experiments.config as conf
# import data_processing.dataset_synapse as dp 
import data_processing.data_parser as dp
import data_processing.dataset_synapse as ds
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf 
import tensorflow_addons as tfa

## Environment Checkup

In [4]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [27]:
importlib.reload(dp)
dr = dp.DataReader("/home/bouzidkenza1/TransUnet/data/synapse-tfrecords/")

In [28]:
dataset = dr.get_dataset()

In [ ]:
for sample in dataset.take(10):
    print(sample[0].shape)
    plt.imshow(sample[0])
    plt.show()

In [19]:
# %%time
# data_dir = "/home/bouzidkenza1/TransUnet/data/train_npz/"
# X, y = dp.load_data(data_dir, 2211)

In [3]:
config = conf.get_b16_none()
trans = vit.TransUnet(config)
trans.compile()

..\models\utils.py:84: UserWarning: Resizing position embeddings from 24 to 32
  UserWarning,


In [12]:
trans.model.layers

In [21]:
data_batch = dataset.batch(30)

In [2]:
%%time
trans.model.fit(data_batch, epochs=3, batch_size=32, verbose=1)

Wall time: 0 ns


## Test Decoder CUP 

In [2]:
X, y = ds.load_data("../data/train_npz/", 10)

In [10]:
X.shape

(10, 512, 512, 3)

In [78]:
importlib.reload(conf)
importlib.reload(vit)
config = conf.get_b16_cup()
trans = vit.TransUnet(config)
trans.compile()

..\models\utils.py:84: UserWarning: Resizing position embeddings from 24 to 32
  UserWarning,


In [81]:
%%time
trans.model.fit(x=X, y=y, epochs=1, batch_size=32, verbose=1)

1/1 [==============================] - 79s 79s/step - loss: 1.8634
Wall time: 1min 19s


## Comparison to B16_None

In [3]:
importlib.reload(conf)
importlib.reload(vit)
config = conf.get_b16_none()
trans = vit.TransUnet(config)
trans.compile()

..\models\utils.py:84: UserWarning: Resizing position embeddings from 24 to 32
  UserWarning,


In [8]:
trans.model.fit(x=X, y=y, epochs=3, batch_size=32, verbose=1)

Epoch 1/3
1/1 [==============================] - 44s 44s/step - loss: 1.6900
Epoch 2/3
1/1 [==============================] - 28s 28s/step - loss: 0.9971
Epoch 3/3
1/1 [==============================] - 27s 27s/step - loss: 0.5585


## Test Resnet

In [10]:
resnet50v2 = tf.keras.applications.ResNet50V2(include_top=False, input_shape=(512,512,3))

In [16]:
resnet50v2.input

<KerasTensor: shape=(None, 512, 512, 3) dtype=float32 (created by layer 'input_7')>

In [59]:
X.shape

(10, 512, 512, 3)

In [62]:
y = resnet50v2(X[0].reshape(1,512,512,3))

In [64]:
y.shape

TensorShape([1, 16, 16, 2048])

In [72]:
# resnet50v2.layers[2].weights

In [11]:
resnet50v2.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 518, 518, 3)  0           input_7[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 256, 256, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 258, 258, 64) 0           conv1_conv[0][0]                 
_________________________________________________________________________________________

In [5]:
weights = np.load("R50+ViT-B_16.npz")


In [14]:
weights.files

['Transformer/encoder_norm/bias',
 'Transformer/encoder_norm/scale',
 'Transformer/encoderblock_0/LayerNorm_0/bias',
 'Transformer/encoderblock_0/LayerNorm_0/scale',
 'Transformer/encoderblock_0/LayerNorm_2/bias',
 'Transformer/encoderblock_0/LayerNorm_2/scale',
 'Transformer/encoderblock_0/MlpBlock_3/Dense_0/bias',
 'Transformer/encoderblock_0/MlpBlock_3/Dense_0/kernel',
 'Transformer/encoderblock_0/MlpBlock_3/Dense_1/bias',
 'Transformer/encoderblock_0/MlpBlock_3/Dense_1/kernel',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/key/bias',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/key/kernel',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/out/bias',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/out/kernel',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/query/bias',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/query/kernel',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/value/bias',
 'Tr

In [7]:
# weights.files
['embedding/kernel', 'embedding/bias']

In [17]:
weights['head/bias'].shape

(21843,)

## Test Hybrid Mode

In [32]:
importlib.reload(conf)
importlib.reload(decoder_layers)
importlib.reload(vit)
importlib.reload(ut)
config = conf.get_b16_hybrid()
trans = vit.TransUnet(config)
trans.compile()

..\models\utils.py:84: UserWarning: Resizing position embeddings from 24 to 8
  UserWarning,


ValueError: Layer weight shape (2, 2, 1024, 768) not compatible with provided weight shape (16, 16, 3, 768)

In [ ]:
%%time
trans.model.fit(x=X, y=y, epochs=1, batch_size=32, verbose=1)

In [93]:
x = tf.ragged.constant([1, 4])
y = tf.constant([2, 5])
z = tf.constant([3, 6])
test = tf.stack([x, y, z])
test

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 4],
       [2, 5],
       [3, 6]])>